In [2]:
import numpy as np
from ase.io import read, write
import spglib
import os

import numpy as np
from ase.io import read
from calorine.calculators import CPUNEP
from calorine.tools import get_force_constants, relax_structure
from matplotlib import pyplot as plt
from pandas import DataFrame
from phonopy.units import THzToCm
from seekpath import get_explicit_k_path
import spglib
from ase.spacegroup.symmetrize import FixSymmetry
from ase.constraints import UnitCellFilter
from ase.optimize import BFGS


In [4]:
# parameters
for n in range(1,11):
    alat = 5.0
    tol = 0.1
    z_spacing = 3.0
    
    atoms_ideal = read('BaZrO3_cubic_2x2x2_with_modes.extxyz')
    atoms_ideal.symbols[atoms_ideal.symbols == 'O'] = 'S'
    
    new_cell = 2 * alat * np.eye(3)
    atoms_ideal.set_cell(new_cell, scale_atoms=True)
    atoms_ideal = atoms_ideal.repeat((1, 1, n))
    
    
    # build slab I
    atoms1 = atoms_ideal.copy()
    mask = atoms1.positions[:, 2] < n * alat + tol
    atoms1 = atoms1[mask]
    new_cell = np.diag([2 * alat, 2 * alat, n * 2 * alat + 2 * z_spacing])
    atoms1.set_cell(new_cell)
    
    # build slab II
    atoms2 = atoms1.copy()
    atoms2.translate([alat/2, alat/2, n * alat + z_spacing])
    
    
    # modes
    modes = ['Mx', 'My', 'Mz', 'Rx', 'Ry', 'Rz']
    for mode in modes:
        atoms1.set_array(mode + '1', atoms1.get_array(mode))
        atoms1.set_array(mode, None)
    
        atoms2.set_array(mode + '2', atoms2.get_array(mode))
        atoms2.set_array(mode, None)
    
    # build full
    atoms_full = atoms1.copy()
    atoms_full.extend(atoms2)
    #view([atoms1, atoms2, atoms_full])
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms_full))
    write(f'RP_{n}_139.in', atoms_full, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms_full, symprec=0.0000001)
    atoms_full.set_constraint(fs)
    atoms_full.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms_full,symprec=0.0000001), atoms_full.get_potential_energy()/atoms_full.get_global_number_of_atoms())
    
    amp = 1


    # apply modes

    # tilt 2 
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += -amp * atoms.get_array('Mz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms), 'MzMz')
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms_full.get_global_number_of_atoms())

    # tilt 3
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Mz1')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms), "")
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 4
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += 0.5 * amp * atoms.get_array('Mz2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 5
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += amp * atoms.get_array('Rz2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 6
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rz1')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 7
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += 0.5 * amp * atoms.get_array('Rz2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 8
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 9
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
   
    # tilt 10
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += 0.5 * amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += 0.5* amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')    
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 11
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
    
    # tilt 12
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    calc = CPUNEP('nep.txt')
    fs = FixSymmetry(atoms, symprec=0.0000001)
    atoms.set_constraint(fs)
    atoms.calc = calc

    ucf = UnitCellFilter(atoms_full)
    dyn = BFGS(ucf, logfile='log.out', trajectory='test.traj')
    dyn.run(fmax=0.0000001, steps=100)
    print(spglib.get_spacegroup(atoms,symprec=0.0000001), atoms.get_potential_energy()/atoms.get_global_number_of_atoms())
   
#    # tilt 13
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += 0.5*amp * atoms.get_array('Ry1')
#    atoms.positions += -amp * atoms.get_array('Rx1')
#    atoms.positions += -0.5*amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    
#    # two tilts 
#    # tilt 14
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Mz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
#    
##    # tilt 15
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Rz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##
#    # tilt 16
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Mz2')
#    traj.append(atoms)
##
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##
##    # tilt 17
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Rz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 18
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 19
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
#
##    # tilt 20
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 21
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 22
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += amp * atoms.get_array('Rx2')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    atoms.positions += amp * atoms.get_array('Mz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 23
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += amp * atoms.get_array('Rx2')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    atoms.positions += amp * atoms.get_array('Rz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 24
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += amp * atoms.get_array('Rx2')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Mz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 25
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += amp * atoms.get_array('Rx2')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Rz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
#
##    # tilt 26
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += -amp * atoms.get_array('Rx2')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    atoms.positions += amp * atoms.get_array('Mz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##    # tilt 27
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += -amp * atoms.get_array('Rx2')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    atoms.positions += amp * atoms.get_array('Rz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##    # tilt 28
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += -amp * atoms.get_array('Rx2')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Mz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##    # tilt 29
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += -amp * atoms.get_array('Rx2')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    atoms.positions += -amp * atoms.get_array('Rz2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##    # tilt 30
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += amp * atoms.get_array('Rx2')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##    # tilt 31
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Mz1')
#    atoms.positions += -amp * atoms.get_array('Rx2')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##    # tilt 32
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += amp * atoms.get_array('Rx2')
#    atoms.positions += amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
##    # tilt 33
#    traj = []
#    atoms = atoms_full.copy()
#    atoms.positions += amp * atoms.get_array('Rx1')
#    atoms.positions += amp * atoms.get_array('Ry1')
#    atoms.positions += amp * atoms.get_array('Rz1')
#    atoms.positions += -amp * atoms.get_array('Rx2')
#    atoms.positions += -amp * atoms.get_array('Ry2')
#    traj.append(atoms)
#
#    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
#    write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')

tilted RP spacegroup: I4/mmm (139)
I4/mmm (139) -2.139058005275299
tilted RP spacegroup: Cmce (64) MzMz
Cmce (64) -1.8124065251350605
tilted RP spacegroup: P4/mbm (127) 
P4/mbm (127) -1.9754762230876952
tilted RP spacegroup: Pbam (55)
Pbam (55) -1.9578844024636672
tilted RP spacegroup: Cmce (64)
Cmce (64) -1.8124065251350605
tilted RP spacegroup: P4/mbm (127)
P4/mbm (127) -1.9754762230876952
tilted RP spacegroup: Pbam (55)
Pbam (55) -1.957884402463667
tilted RP spacegroup: P4_2/nnm (134)
P4_2/nnm (134) -1.80772549303113
tilted RP spacegroup: Cccm (66)
Cccm (66) -0.41683203043171485
tilted RP spacegroup: Pnnn (48)
Pnnn (48) -1.5472774349719292
tilted RP spacegroup: P4_2/ncm (138)
P4_2/ncm (138) -1.8724325995099267
tilted RP spacegroup: Cmce (64)
Cmce (64) -0.5614593845242428
tilted RP spacegroup: I4/mmm (139)
I4/mmm (139) -2.095630619592044
tilted RP spacegroup: Cmce (64) MzMz
Cmce (64) -1.7951794659820866
tilted RP spacegroup: P4/mbm (127) 
P4/mbm (127) -1.9452901118147148
tilted RP sp